In [1]:
from transformers import pipeline
import os
# from dask.distributed import Client
from pathlib import Path
import re

pipe = pipeline("feature-extraction", model="microsoft/codebert-base")

### Why CodeBert?
Codebert is a model trained on both comments (of the code) and code. [Model](https://huggingface.co/microsoft/codebert-base). [Data](https://github.com/github/CodeSearchNet#data).

In [2]:
vector = pipe(["import os", "a"])
len(vector), len(vector[0]), len(vector[0][0])

(2, 1, 4)

# Job

In [3]:
path_dir = "./input_data/"

In [4]:
all_files = [Path(path_dir + x) for x in os.listdir(path_dir)]
print(all_files[:3])

[WindowsPath('input_data/ClimbingStairs.py'), WindowsPath('input_data/Containers.py'), WindowsPath('input_data/HouseRobberDynProg.py')]


In [5]:
def get_texts(filepath):
    with open(filepath) as file:  
        return file.read()

In [6]:
corpus = [get_texts(path) for path in all_files]
embeddings = [out for out in pipe(corpus, padding=True, truncation=True)]
print(
    len(embeddings), len(embeddings[0]), len(embeddings[0][0]), len(embeddings[0][0][0])
)
cls_components = [x[0][0] for x in embeddings]
print(len(cls_components), len(cls_components[0]))

12 1 192 768
12 768


In [7]:
import pandas as pd

df = pd.DataFrame({"text": corpus, "vector": cls_components})

df.to_parquet("output_data/dataframe_text_embeddings.parquet")